In [1]:
import os
os.chdir('/Users/layne/Desktop/aiqc')
os.getcwd()

'/Users/layne/Desktop/aiqc'

In [2]:
import aiqc
from aiqc import examples

/Users/layne/.pyenv/versions/3.7.6/envs/jupyterlab/lib/python3.7/site-packages/pandas/compat/__init__.py:97: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [3]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import *

In [4]:
from importlib import reload
aiqc.delete_db(True)
reload(aiqc)
aiqc.create_db()


=> Success - deleted database file at path:
/Users/layne/Library/Application Support/aiqc/aiqc.sqlite3


=> Success - created database file at path:
/Users/layne/Library/Application Support/aiqc/aiqc.sqlite3


=> Success - created the following tables within database:
['algorithm', 'batch', 'dataset', 'encoderset', 'featurecoder', 'featureset', 'file', 'fold', 'foldset', 'hyperparamcombo', 'hyperparamset', 'image', 'job', 'jobset', 'label', 'labelcoder', 'result', 'splitset', 'tabular']



---

In [6]:
dataset = aiqc.Dataset.Tabular.from_path(
    examples.get_demo_file_path('exoplanets.parquet')
    , source_file_format = 'parquet'
)

In [7]:
dataset.to_pandas().head().dtypes

TypeFlag                  int64
PlanetaryMassJpt        float64
PeriodDays              float64
SurfaceTempK            float64
DistFromSunParsec       float64
HostStarMassSlrMass     float64
HostStarRadiusSlrRad    float64
HostStarMetallicity     float64
HostStarTempK           float64
dtype: object

In [14]:
dataset.to_pandas().tail(5)

,TypeFlag,PlanetaryMassJpt,PeriodDays,SurfaceTempK,DistFromSunParsec,HostStarMassSlrMass,HostStarRadiusSlrRad,HostStarMetallicity,HostStarTempK
428,0,0.220,6.212290,783.1,380.0,0.880,0.770,0.30,5080.0
429,0,4.063,4.124461,1161.5,254.0,0.947,1.105,0.01,5588.0
430,0,3.328,5.452700,1335.3,257.0,1.392,1.535,0.22,6442.0
431,0,1.360,395.400000,256.8,68.5,1.000,1.220,0.14,5407.0
432,0,1.018,1605.800000,160.2,68.5,1.000,1.220,0.14,5407.0


In [15]:
label = dataset.make_label(columns=['SurfaceTempK'])

In [16]:
featureset = dataset.make_featureset(exclude_columns=[
    'SurfaceTempK',
])

In [17]:
splitset = featureset.make_splitset(
    label_id=label.id
    , size_test = 0.22
    , size_validation = 0.12 
)

In [18]:
foldset = splitset.make_foldset(
    fold_count = 3
)

---

In [19]:
encoderset = splitset.make_encoderset()

In [11]:
# labelcoder = encoderset.make_labelcoder(
#     sklearn_preprocess = OneHotEncoder(sparse=False)
#     , only_fit_train = False
# )

In [20]:
labelcoder = encoderset.make_labelcoder(
    sklearn_preprocess = StandardScaler()
    , only_fit_train = True
)

---
---
---

In [21]:
fc = encoderset.make_featurecoder(
	sklearn_preprocess = OneHotEncoder(sparse=False)
	, only_fit_train = False
	, include = True
	, dtypes = ['int64']
	, columns = None
)

In [22]:
fc = encoderset.make_featurecoder(
	sklearn_preprocess = StandardScaler()
	, only_fit_train = True
	, include = True
	, dtypes = ['float64']
	, columns = None
)

In [189]:
# fc = encoderset.make_featurecoder(
# 	sklearn_preprocess = OneHotEncoder(sparse=False)
# 	, only_fit_train = False
# 	, include = True
# 	, dtypes = ['uint8', 'int64']
# 	, columns = None
# )

---
---
---

In [24]:
import keras
from keras import metrics
from keras.models import Sequential
from keras.callbacks import History
from keras.layers import Dense, Dropout

In [25]:
len(featureset.columns)

8

In [26]:
def function_model_build(**hyperparameters):
    model = Sequential()
    
    model.add(Dense(hyperparameters['neuron_count'], activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(hyperparameters['neuron_count'], activation='relu'))
    
    if hyperparameters['2nd_dropout']:
        model.add(Dropout(0.2))
        
    model.add(Dense(1, kernel_initializer='normal'))
    
    
    opt = keras.optimizers.Adamax(hyperparameters['learning_rate'])
    model.compile(
		loss='mean_squared_error'
		, optimizer=opt
	)
    return model

In [27]:
def function_model_train(model, samples_train, samples_evaluate, **hyperparameters):   
    
    metrics_cuttoffs = [
        {"metric":"val_loss", "cutoff":0.3, "above_or_below":"below"},
        {"metric":"loss", "cutoff":0.3, "above_or_below":"below"}
    ]
    cutoffs = aiqc.TrainingCallback.Keras.MetricCutoff(metrics_cuttoffs)
    
    model.fit(
        samples_train["features"]
        , samples_train["labels"]
        , validation_data = (
            samples_evaluate["features"]
            , samples_evaluate["labels"]
        )
        , verbose = 0
        , batch_size = hyperparameters['batch_size']
        , callbacks=[History(), cutoffs]
        , epochs = hyperparameters['epoch_count']
    )

    return model

In [28]:
algorithm = aiqc.Algorithm.make(
    library = "keras"
    , analysis_type = "regression"
    , function_model_build = function_model_build
    , function_model_train = function_model_train
)

In [29]:
hyperparameters = {
    "batch_size": [3, 5]
    , "2nd_dropout": [True, False]
    , "epoch_count": [500]
    , "neuron_count": [32, 64]
    , "learning_rate": [0.01]
}

In [31]:
hyperparamset = aiqc.Hyperparamset.from_algorithm(
    algorithm_id = algorithm.id
    , hyperparameters = hyperparameters
)

In [32]:
batch = aiqc.Batch.from_algorithm(
    algorithm_id = algorithm.id
    , splitset_id = splitset.id
    , hyperparamset_id = hyperparamset.id
    , foldset_id = None
    , encoderset_id = encoderset.id
)

In [33]:
batch.run_jobs()

🔮 Training Models 🔮:   0%|                                                  | 0/8 [00:00<?, ?it/s]


:: Epoch #158 ::
Satisfied thresholds defined in `MetricCutoff` callback. Stopped training early.



🔮 Training Models 🔮:  12%|█████▎                                    | 1/8 [00:35<04:06, 35.19s/it]


:: Epoch #105 ::
Satisfied thresholds defined in `MetricCutoff` callback. Stopped training early.



🔮 Training Models 🔮:  25%|██████████▌                               | 2/8 [01:02<03:16, 32.82s/it]


:: Epoch #167 ::
Satisfied thresholds defined in `MetricCutoff` callback. Stopped training early.



🔮 Training Models 🔮:  38%|███████████████▊                          | 3/8 [01:33<02:41, 32.22s/it]


:: Epoch #99 ::
Satisfied thresholds defined in `MetricCutoff` callback. Stopped training early.



🔮 Training Models 🔮:  50%|█████████████████████                     | 4/8 [01:53<01:53, 28.26s/it]
Process aiqc_batch_1:
Traceback (most recent call last):
  File "/Users/layne/.pyenv/versions/3.7.6/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/Users/layne/.pyenv/versions/3.7.6/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/layne/Desktop/aiqc/aiqc/__init__.py", line 3187, in background_proc
    job.run(verbose=verbose, repeat_index=repeat_index)
  File "/Users/layne/Desktop/aiqc/aiqc/__init__.py", line 3736, in run
    , **hyperparameters
  File "<ipython-input-27-ea8b7976d0ab>", line 19, in function_model_train
    , epochs = hyperparameters['epoch_count']
  File "/Users/layne/.pyenv/versions/3.7.6/envs/jupyterlab/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py", line 108, in _method_wrapper
    return method(self, *args, **kwargs)
  File "/Users